In [30]:
import pandas as pd

# Load data
news_final = pd.read_csv("/Users/sumanur/Desktop/ml/octeight.csv") 
news_final = news_final.dropna()
news_final = news_final.drop(columns=['SentimentTitle', 'SentimentHeadline', 'IDLink'])
news_final = news_final.drop(columns=['GooglePlus'])
news_final = news_final.drop(columns=['Facebook'])
news_final = news_final.drop(columns=['Topics_lda'])
news_final = news_final.drop(columns=['Topics_lda_title'])

In [31]:
news_final

,Title,Headline,Source,Topic,PublishDate,LinkedIn
0,"On The Economy, Give Obama AD",Hillary Clinton got the laugh line of the week...,Forbes,economy,2015-11-09 22:40:10,2
1,"To one CEO, 'Janet Yellen's Magic Money Machin...","""""""I believe something is shifting in the unde...",MarketWatch,economy,2015-11-09 22:40:10,0
2,3 Ways to Jumpstart a Community-Grounded Economy,"This article is part of New Economy Week, a co...",YES! Magazine,economy,2015-11-09 09:40:10,0
3,"Despite China Slowdown, Global Economy Expecte...",Global trade and economic activity--held back ...,International Business Times,economy,2015-11-09 11:40:10,10
4,New Economy Week Starts Here,"New Economy Week Starts Here. This week, we br...",YES! Magazine,economy,2015-11-09 09:40:10,0
5,We Must Protect the On-Demand Economy to Prote...,This has never been more apparent than in the ...,WIRED,economy,2015-11-09 18:40:10,129
6,Poll finds most Californians dissatisfied with...,LOS ANGELES (Reuters) - Sixty percent of Calif...,Yahoo News,economy,2015-11-09 23:40:10,0
7,EconomyGoldman warns of risk of US economic ov...,Despite dreary international economic data and...,Financial Times,economy,2015-11-09 16:40:10,0
8,Egypt's Economy: The Next Casualty Of The Metr...,"It's not grabbing the most headlines, for obvi...",Fortune,economy,2015-11-09 12:40:10,12
9,Why Kazakhstan holds the keys to the global ec...,"Russia, having challenged the current internat...",The Independent,economy,2015-11-09 15:40:10,46


In [4]:
# Sentiment Analysis 
headlines = list(news_final['Headline'])
titles = list(news_final['Title'])
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
sia = SentimentIntensityAnalyzer()

# Adds headline sentiment to the dataframe
headline_sentiment = []
for h in headlines:
    hs = sia.polarity_scores(h)
    headline_sentiment.append(hs)
compound_headline_sentiments = []
for hs in headline_sentiment:
    compound_headline_sentiments.append(hs['compound'])
    
compound_headline_sentiments = pd.Series(compound_headline_sentiments)
news_final['HeadlineSentiment'] = compound_headline_sentiments.values

# Adds title sentiment to the dataframe
title_sentiment = []
for t in titles:
    ts = sia.polarity_scores(t)
    title_sentiment.append(ts)
compound_title_sentiments = []
for ts in title_sentiment:
    compound_title_sentiments.append(ts['compound'])
    
compound_title_sentiments = pd.Series(compound_title_sentiments)
news_final['TitleSentiment'] = compound_title_sentiments.values

In [5]:
# Remove time stamp and add 'Hour of Day' column and 'Day of Week' column
import datetime
news_dates = news_final['PublishDate']

hour_of_day = []
for date in news_dates:
    news_date =  datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    news_hour = news_date.hour
    hour_of_day.append(news_hour)
news_final['HourOfDay'] = hour_of_day
news_final['HourOfDay'] = news_final['HourOfDay']//3

day_of_week = []
for date in news_dates:
    news_date =  datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    news_day = news_date.weekday()
    day_of_week.append(news_day)
news_final['DayOfWeek'] = day_of_week

# Discard
news_final = news_final.drop(columns=['PublishDate'])
news_final = news_final.drop(columns=['Title'])
news_final = news_final.drop(columns=['Headline'])
news_final = news_final.drop(columns=['Source'])

In [6]:
# 0 to 3 topic labels
topic_labels = []
for e in news_final['Topic']:
    if e == 'economy':
        topic_labels.append(0)
    if e == 'obama':
        topic_labels.append(1)
    if e == 'palestine':
        topic_labels.append(2)
    if e == 'microsoft':
        topic_labels.append(3)

news_final['TopicLabels'] = topic_labels
news_final = news_final.drop(columns=['Topic'])

# Scaling
news_final['HourOfDay'] = news_final['HourOfDay']/7
news_final['DayOfWeek'] = news_final['DayOfWeek']/6
news_final['TopicLabels'] = news_final['TopicLabels']/3

In [7]:
# Add K means column
from sklearn.cluster import KMeans
import numpy as np

X = np.array(news_final['LinkedIn'])
X = X.reshape(-1,1)

kk = 2
kmeans = KMeans(n_clusters=kk, random_state=0, max_iter=100, algorithm="full", ).fit(X)
news_final['FBPopMean'] = kmeans.predict(X)

# Cluster based oversampling
m0 = news_final[news_final.FBPopMean == 0]
m1 = news_final[news_final.FBPopMean == 1]

m0_final = m0
m1_final = m1

while len(m1_final) < len(m0):
    m1_final = m1_final.append(m1)

news_final_mega = m0_final.append(m1_final)
news_final = news_final_mega

In [29]:
news_final.ix[80000]

/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


LinkedIn             0.000000
HeadlineSentiment    0.000000
TitleSentiment      -0.296000
HourOfDay            0.571429
DayOfWeek            0.666667
TopicLabels          0.333333
FBPopMean            0.000000
Name: 80000, dtype: float64

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from sklearn.preprocessing import OneHotEncoder

# Prepare data for training
enc = OneHotEncoder(handle_unknown='ignore')
Y = enc.fit_transform(np.array(news_final['FBPopMean']).reshape(-1,1))
X = np.array(news_final.drop(columns=['LinkedIn','FBPopMean']))

# Define model
model = Sequential([
    Dense(8, input_shape=(5,)),Activation('relu'),
    Dense(6),Activation('sigmoid'),
    Dense(kk),Activation('softmax'),
])
model.compile(optimizer='nadam', #rmsprop 60 adamax 80
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Training
model.fit(X,Y, epochs=20)

Epoch 1/21
185883/185883 [==============================] - 11s 59us/step - loss: 0.1109 - acc: 0.9693 1s - los
Epoch 2/21
185883/185883 [==============================] - 11s 59us/step - loss: 0.0579 - acc: 0.9869
Epoch 3/21
185883/185883 [==============================] - 11s 57us/step - loss: 0.0489 - acc: 0.9892
Epoch 4/21
185883/185883 [==============================] - 10s 56us/step - loss: 0.0449 - acc: 0.9904
Epoch 5/21
185883/185883 [==============================] - 11s 57us/step - loss: 0.0423 - acc: 0.9911
Epoch 6/21
185883/185883 [==============================] - 9s 51us/step - loss: 0.0405 - acc: 0.9914
Epoch 7/21
185883/185883 [==============================] - 10s 54us/step - loss: 0.0391 - acc: 0.9919
Epoch 8/21
185883/185883 [==============================] - 10s 53us/step - loss: 0.0379 - acc: 0.9922
Epoch 9/21
185883/185883 [==============================] - 10s 55us/step - loss: 0.0366 - acc: 0.9924
Epoch 10/21
185883/185883 [==============================] - 11s 

In [13]:
# Logistic Regression
xx =X
yy = news_final['FBPopMean']
yy = np.array(yy)
xx
from sklearn.model_selection import train_test_split
xx_train, xx_test, yy_train, yy_test = train_test_split(xx, yy, test_size=0.25, random_state=0)

from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(max_iter=1000)
logisticRegr.fit(xx_train, yy_train)
print("Logistic Regression Accuracy = ")
logisticRegr.score(xx_test, yy_test)

Logistic Regression Accuracy = 


0.9665382711798757

In [14]:
# Correlation scores
print("Correlation Scores")
print(news_final['FBPopMean'].corr(news_final['TopicLabels']))
print(news_final['FBPopMean'].corr(news_final['HeadlineSentiment']))
print(news_final['FBPopMean'].corr(news_final['TitleSentiment']))
print(news_final['FBPopMean'].corr(news_final['HourOfDay']))
print(news_final['FBPopMean'].corr(news_final['DayOfWeek']))

Correlation Scores
0.6448861898534324
0.39761052008789255
0.0898819630470705
0.035604611398118706
-0.6907628589628124
